In [1]:
import pandas as pd
import csv 

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
dataFormations = pd.read_csv('src/moncompteformation_formations_engagees.csv', sep=';', low_memory=False)

with open('src/moncompteformation-les-usagers.csv', 'r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    valid_rows = []
    for row in reader:
        try:
            valid_rows.append(row)
        except Exception as e:
            print(f"Skipping a row due to error: {e}")

dataUsagers = pd.DataFrame(valid_rows[1:], columns=valid_rows[0])


In [3]:
# import pandas
import pandas as pd

# Load the data
dataFormations = pd.read_csv('src/moncompteformation_formations_engagees.csv', sep=';', low_memory=False)

# Group the data
grouped = dataFormations.groupby(['code_rs', 'code_rncp', 'departement_lieu_formation', 'date validation'])

# Create a new DataFrame
new_data = pd.DataFrame(grouped.size().reset_index(name = "Group_Count"))

# Create a new id for each row
new_data['id_dossier'] = new_data.index + 1

# Get the unique status
unique_status = dataFormations['statut_dossier'].unique()

# Create a new column for each unique status
for status in unique_status:
    new_data[status] = dataFormations['statut_dossier'].apply(lambda x: 1 if x == status else 0)


In [4]:
# Expand rows where Group_Count is greater than 1
new_data = new_data.loc[new_data.index.repeat(new_data['Group_Count'])]

# Update Group_Count to 1 for all rows
new_data['Group_Count'] = 1

# Reset the index to get new ids, and drop the old index
new_data.reset_index(drop=True, inplace=True)

# Create a new id for each row
new_data['id'] = new_data.index + 1


In [5]:
new_data.head(5)
new_data.columns

Index(['code_rs', 'code_rncp', 'departement_lieu_formation', 'date validation',
       'Group_Count', 'id_dossier', 'Clos - Réalisation Totale',
       'Clos - Réalisation Partielle', 'Clos - Non Réalisé annulé titulaire',
       'Clos - Non Réalisé annulé OF', 'Validé', 'Sortie de formation',
       'Entrée en formation', 'Rejeté', 'Dossier annulé gestionnaire', 'id'],
      dtype='object')

In [6]:
new_data = new_data.rename(columns={
    'code_rs': 'code_rs', 
    'code_rncp': 'code_rncp', 
    'departement_lieu_formation': 'departement_lieu_formation', 
    'date validation': 'date validation',
    'Group_Count': 'Group_Count', 
    'id_dossier': 'id', 
    'Clos - Réalisation Totale': 'r.tot',
    'Clos - Réalisation Partielle': 'r.part', 
    'Clos - Non Réalisé annulé titulaire': 'an.tit',
    'Clos - Non Réalisé annulé OF': 'an.OF', 
    'Validé': 'val', 
    'Sortie de formation': 'sort',
    'Entrée en formation': 'ent', 
    'Rejeté': 'rej', 
    'Dossier annulé gestionnaire': 'DAG'
})

In [7]:
new_data.head(5)


,code_rs,code_rncp,departement_lieu_formation,date validation,Group_Count,id,r.tot,r.part,an.tit,an.OF,val,,EF,R,DAG,id
0,-2,-2,Ain,2020-05,1,1,1,0,0,0,0,0,0,0,0,1
1,-2,-2,Ain,2020-05,1,1,1,0,0,0,0,0,0,0,0,2
2,-2,-2,Aisne,2020-01,1,2,1,0,0,0,0,0,0,0,0,3
3,-2,-2,Allier,2020-01,1,3,1,0,0,0,0,0,0,0,0,4
4,-2,-2,Allier,2020-06,1,4,1,0,0,0,0,0,0,0,0,5


In [10]:
# import necessary libraries
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

# Define the target variable and the features
features = new_data.columns[6:]  # all columns after 'id'
target = new_data.columns[-1]  # assuming the last column is the target

X = new_data[features]
y = new_data[target]

# Add intercept to input variables
X = sm.add_constant(X)

# Fit the logistic regression model
model_logistic = sm.GLM(y, X, family=sm.families.Binomial())
result = model_logistic.fit()

# Get the odds ratio
odds_ratio = np.exp(result.params)

# Random Forest model
model_rf = RandomForestClassifier(n_estimators=10, random_state=42)
model_rf.fit(X, y)


# Get feature importance
feature_importance = model_rf.feature_importances_


MemoryError: could not allocate 2475452416 bytes

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Define the target variable and the features
features = new_data.columns[6:]  # all columns after 'id'
target = new_data.columns[-1]  # assuming the last column is the target

X = new_data[features]
y = new_data[target]

# Fit the logistic regression model
model_logistic = LogisticRegression()
model_logistic.fit(X, y)

# Get the coefficients
coefficients = model_logistic.coef_

# Get the odds ratio
odds_ratio = np.exp(coefficients)

# Random Forest model
model_rf = RandomForestClassifier()
model_rf.fit(X, y)

# Get feature importance
feature_importance = model_rf.feature_importances_


ValueError: y should be a 1d array, got an array of shape (1208717, 2) instead.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Define the target variable and the features
features = new_data.columns[6:]  # all columns after 'id'
target = new_data.columns[-1]  # assuming the last column is the target

X = new_data[features]
y = new_data[target]

# Fit the logistic regression model
model_logistic = LogisticRegression()
model_logistic.fit(X, y)

# Get the coefficients
coefficients = model_logistic.coef_

# Get the odds ratio
odds_ratio = np.exp(coefficients)

# Get the predictions and add them to the DataFrame
new_data['predictions'] = model_logistic.predict(X)

# Random Forest model
model_rf = RandomForestClassifier()
model_rf.fit(X, y)

# Get feature importance
feature_importance = model_rf.feature_importances_


ValueError: y should be a 1d array, got an array of shape (1208717, 2) instead.

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

# add proportion test column (replace 'group1' and 'group2' with your actual groups)
group1 = new_data[new_data['code_rs'] == 1]  # Change this with actual condition
group2 = new_data[new_data['code_rncp'] == 1]  # Change this with actual condition

count = np.array([len(group1), len(group2)])
nobs = len(new_data)

stat, pval = stats.proportions_ztest(count, nobs)
new_data['proportion_test'] = pval

# features for regression
features = ['r.tot', 'r.part', 'an.tit', 'an.OF']
X = new_data[features]

# target variable - replace 'target' with your actual target column
y = new_data['target'] 

# Add intercept to input variables
X = sm.add_constant(X)

# Fit the logistic regression model
model_logistic = sm.Logit(y, X)
result = model_logistic.fit()

# Add the predictions to a new column
new_data['predictions'] = result.predict(X)


AttributeError: module 'scipy.stats' has no attribute 'proportions_ztest'

In [19]:
# Import necessary libraries
from sklearn.preprocessing import OneHotEncoder

# Define the input features
input_features = ['code_rs', 'code_rncp', 'departement_lieu_formation', 'date validation']
X = new_data[input_features]

# Create an encoder object
encoder = OneHotEncoder(drop='first')

# Fit and transform the data
X_encoded = encoder.fit_transform(X)

# Convert to dataframe
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names(input_features))

# Define the output features
output_features = ['r.tot', 'r.part', 'an.tit', 'an.OF']

# For each output feature, fit a logistic regression model
for feature in output_features:
    y = new_data[feature]
    model = sm.Logit(y, X_encoded)
    result = model.fit()
    # Here, you can do something with the result, e.g. print the summary
    print(result.summary())


# Fit the logistic regression model for 'r.part'
y_rpart = new_data['r.part']
model_rpart = sm.Logit(y_rpart, X)
result_rpart = model_rpart.fit()

# Add the predictions to a new column
new_data['predicted_r.part'] = result_rpart.predict(X)

# Fit the logistic regression model for 'an.tit'
y_antit = new_data['an.tit']
model_antit = sm.Logit(y_antit, X)
result_antit = model_antit.fit()

# Add the predictions to a new column
new_data['predicted_an.tit'] = result_antit.predict(X)

# Fit the logistic regression model for 'an.OF'
y_anof = new_data['an.OF']
model_anof = sm.Logit(y_anof, X)
result_anof = model_anof.fit()

# Add the predictions to a new column
new_data['predicted_an.OF'] = result_anof.predict(X)


MemoryError: Unable to allocate 52.0 GiB for an array with shape (1208717, 5773) and data type float64

In [16]:
correlation_matrix = new_data[features].corr()
print(correlation_matrix)


           r.tot    r.part    an.tit     an.OF       val                  EF  \
r.tot   1.000000 -0.369482 -0.468853 -0.422964 -0.171132 -0.082352 -0.290593   
r.part -0.369482  1.000000 -0.120559 -0.108759 -0.044004 -0.021176 -0.074722   
an.tit -0.468853 -0.120559  1.000000 -0.138010 -0.055839 -0.026871 -0.094818   
an.OF  -0.422964 -0.108759 -0.138010  1.000000 -0.050374 -0.024241 -0.085538   
val    -0.171132 -0.044004 -0.055839 -0.050374  1.000000 -0.009808 -0.034609   
       -0.082352 -0.021176 -0.026871 -0.024241 -0.009808  1.000000 -0.016654   
EF     -0.290593 -0.074722 -0.094818 -0.085538 -0.034609 -0.016654  1.000000   
R      -0.003448 -0.000887 -0.001125 -0.001015 -0.000411 -0.000198 -0.000697   
DAG    -0.002181 -0.000561 -0.000712 -0.000642 -0.000260 -0.000125 -0.000441   
id      0.003620 -0.003460 -0.095062  0.007725  0.074806  0.027454  0.072788   
id      0.002113 -0.012904 -0.106739  0.026759  0.075213  0.028955  0.078154   

               R       DAG        id   

In [17]:
print(new_data[features].nunique())


r.tot           2
r.part          2
an.tit          2
an.OF           2
val             2
                2
EF              2
R               2
DAG             2
id         310010
id        1208717
dtype: int64


In [22]:
# import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

# Define the target variables and the features
input_features = ['code_rs', 'code_rncp', 'departement_lieu_formation', 'date validation']
target_features = ['r.tot', 'r.part', 'an.tit', 'an.OF']

X = new_data[input_features]
y = new_data[target_features]

# One-hot encoding for input features
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
X_encoded = encoder.fit_transform(X)

# Create separate models for each target feature
models = {}
for target in target_features:
    model = LogisticRegression(solver='saga', max_iter=5000)
    model.fit(X_encoded, new_data[target])
    models[target] = model

# Now models dictionary contains a separate model for each target feature


In [27]:
# import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Define the target variables and the features
input_features = ['code_rs', 'code_rncp', 'departement_lieu_formation', 'date validation']
target_features = ['r.tot', 'r.part', 'an.tit', 'an.OF']

# Split the data into training set and test set
train_data, test_data = train_test_split(new_data, test_size=0.2, random_state=42)

X_train = train_data[input_features]
y_train = train_data[target_features]
X_test = test_data[input_features]
y_test = test_data[target_features]

# One-hot encoding for input features
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train)

# Create separate models for each target feature
models = {}
for target in target_features:
    model = LogisticRegression(solver='saga', max_iter=5000)
    model.fit(X_train_encoded, y_train[target])
    models[target] = model

# Transform the test data using the same encoder
X_test_encoded = encoder.transform(X_test)

# Predict and evaluate the models
for target, model in models.items():
    y_pred = model.predict(X_test_encoded)
    accuracy = accuracy_score(y_test[target], y_pred)
    print(f'Accuracy for predicting {target}: {accuracy}')


Accuracy for predicting r.tot: 0.7587737441260176
Accuracy for predicting r.part: 0.9115221060295188
Accuracy for predicting an.tit: 0.9127299953669998
Accuracy for predicting an.OF: 0.8878400291217156


In [21]:
print(new_data.columns)


Index(['code_rs', 'code_rncp', 'departement_lieu_formation', 'date validation',
       'Group_Count', 'id', 'r.tot', 'r.part', 'an.tit', 'an.OF', 'val', '',
       'EF', 'R', 'DAG', 'id'],
      dtype='object')
